In [1]:
%load_ext autoreload
%autoreload 2
from gui import SpikeImportGUI
from importers import SpikeImporter
from recordings import MNGRecording

# present the Spike GUI and let the user make his choices
spike_gui = SpikeImportGUI()

importer = SpikeImporter(filepath = spike_gui.filepath, time_channel = spike_gui.time_channel, signal_channel = spike_gui.signal_channel)

recording = MNGRecording.from_importer(importer = importer, stimulus_channels = spike_gui.stimulus_channels, \
                                      ap_channels = spike_gui.ap_channels, force_threshold = spike_gui.force_threshold, \
                                      max_ap_gap_time = spike_gui.max_ap_gap)
_ = recording.split_into_sweeps()

List of eletrical stimuli created.
Finished processing AP channel 1 out of 1
List of APs created.
Done with cropping the intervals


In [24]:
'''
Here, we extract some features from the APs:
- postpulse distance (aka latency)
- normalized signal energy
'''
%autoreload 2
from tqdm import tqdm
from feature_extraction import ResponseLatency, NormalizedSignalEnergy, SpikeCount, AdaptiveSpikeCount

print("Extracting features from every AP")
features = [ResponseLatency(regular_el_stimuli = recording.el_stimuli), \
            NormalizedSignalEnergy(), \
            SpikeCount(actpots = recording.actpots, timeframe = 100, num_intervals = 16), \
            AdaptiveSpikeCount(actpots = recording.actpots, timeframe = 100, num_splits = 8)]

for ap in tqdm(recording.actpots):
    # for each of the features, calculate and store in the ap's dict
    for feature in features:
        ap.features[feature.get_feature_name()] = feature.get_feature_value(ap)

    # also, save a reference to the previous electrical stimulus.
    # we might need that later.
    ap.prev_stimuli["regular"] = MNGRecording.get_prev_el_stimulus(ap, recording.el_stimuli)

  1%|▌                                                                                | 8/1272 [00:00<00:17, 73.42it/s]

Extracting features from every AP


100%|██████████████████████████████████████████████████████████████████████████████| 1272/1272 [00:13<00:00, 91.27it/s]


In [29]:
%autoreload 2
from plotting import FallingLeafPlot
from ipywidgets import interact_manual, fixed, IntSlider, FloatSlider

tmin, tmax = importer.get_time_range()
max_interval_length = max([stim.interval_length for stim in recording.el_stimuli])

flplot = FallingLeafPlot()
interact_manual(flplot.plot, regular_stimuli = fixed(recording.el_stimuli), action_potentials = fixed(recording.actpots), \
                t_start = FloatSlider(min = tmin, max = tmax, value = 0), \
                num_intervals = IntSlider(min = 1, max = len(recording.el_stimuli), step = 1, value = 300), \
                post_stimulus_timeframe = FloatSlider(min = 0, max = max_interval_length, step = 0.01, value = 0.05), \
                ap_tracks = fixed(recording.ap_tracks), \
                manual_name = "Update Plot")

interactive(children=(FloatSlider(value=0.0, description='t_start', max=300.0), IntSlider(value=300, descripti…

<function ipywidgets.widgets.interaction._InteractFactory.__call__.<locals>.<lambda>(*args, **kwargs)>

In [3]:
''' 
This cell performs the clustering according to
- distance to previous stimulus
- normalized signal energy of the APs
'''
%autoreload 2
from fibre_tracking import DBSCANClustering
from ipywidgets import interact_manual, fixed, IntSlider, FloatSlider

# wipe all the fibre indices
for ap in recording.actpots:
    ap.implied_fibre_index = None
    
pure_track_aps = [ap for ap in recording.actpots if ap.onset < 50]

dbscan = DBSCANClustering()
x = interact_manual(dbscan.perform_clustering, actpots = fixed(pure_track_aps), \
                eps = FloatSlider(min = 0.00005, max = 0.005, step = 0.00001, value = 0.001, readout_format='.5f'), \
                min_samples = IntSlider(min = 1, max = 50, value = 10, step = 1), \
                save_fibre_prediction = fixed(True), \
                plot_results = fixed(True), \
                manual_name = "Update Plot")

interactive(children=(FloatSlider(value=0.001, description='eps', max=0.005, min=5e-05, readout_format='.5f', …

In [16]:
%autoreload 2
from fibre_tracking.ap_track import APTrack
from plotting import FallingLeafPlot
from tqdm import tqdm

ap_tracks = []

num_intv = min(50, len(recording.sweeps) - 3)

for cluster_index in [0]:
    aps = [ap for ap in pure_track_aps if ap.implied_fibre_index == cluster_index]
    ap_track = APTrack.from_aps(sweeps = recording.sweeps, aps = aps)
    ap_track.extend_downwards(recording.sweeps, num_sweeps = 300, max_shift = 0.003, radius = 2, verbose = False)
    ap_tracks.append(ap_track)
    
t_start = 0


flplot = FallingLeafPlot()
flplot.plot(regular_stimuli = recording.el_stimuli, action_potentials = recording.actpots, t_start = t_start, num_intervals = 500, \
            ap_tracks = ap_tracks, post_stimulus_timeframe = 0.025, plot_raw_signal= False)

In [20]:
%autoreload 2
print(ap_tracks[0])

aps = ap_tracks[0].get_nearest_existing_aps(sweeps = recording.sweeps)

for ap in aps:
    ap.implied_fibre_index = 0
    
flplot = FallingLeafPlot()
flplot.plot(regular_stimuli = recording.el_stimuli, action_potentials = recording.actpots, t_start = t_start, num_intervals = 500, \
            ap_tracks = ap_tracks, post_stimulus_timeframe = 0.025, plot_raw_signal = False)

In [42]:
%autoreload 2

from fibre_tracking import APTrack

track1 = recording.ap_tracks[0]
track2 = APTrack([])

new_track = APTrack.merge_tracks(track1, track2)


In [46]:
new_track.save_to_csv(dir = "../data", fname = "test_track.csv")

Successfully saved AP track to ../data\test_track.csv


In [49]:
very_new_track = APTrack.load_from_csv("../data/test_track.csv")

In [50]:
print(very_new_track.latencies)

[0.010250000000000002, 0.010275000000000034, 0.010275000000000034, 0.01032499999999992, 0.010224999999999929, 0.010275000000000034, 0.010224999999999929, 0.010275000000000034, 0.010275000000000034, 0.010224999999999708, 0.010224999999999708, 0.01022500000000015, 0.01022500000000015, 0.010299999999999976, 0.010174999999999823, 0.01024999999999987, 0.010200000000000651, 0.01027499999999959, 0.010325000000000362, 0.010224999999999708, 0.01027499999999959, 0.010275000000000478, 0.01027499999999959, 0.010275000000000478, 0.010224999999999708, 0.01027499999999959, 0.010225000000000595, 0.010224999999999708, 0.010225000000000595, 0.010249999999999648, 0.010250000000000536, 0.010199999999999765, 0.010325000000000806, 0.010274999999999146, 0.010275000000000034, 0.01022500000000015, 0.010224999999999262, 0.01022500000000015, 0.010250000000000092, 0.010274999999999146, 0.010275000000000034, 0.01022500000000015, 0.010275000000000922, 0.010275000000000034, 0.010299999999999976, 0.01025000000000098,